In [ ]:
import numpy as np
import os
from sklearn.decomposition import PCA, TruncatedSVD
import cv2
from scipy.signal import convolve2d
from sklearn.impute import KNNImputer
from scipy.ndimage import median_filter
from tqdm import tqdm


In [ ]:
def preprocess_images(batch_size,base_dir, target_size=(64, 64)):
    processed_images = []
    img_dir = os.listdir(base_dir)
    
    for i in tqdm(range(0, len(img_dir), batch_size)):
        batch_files = img_dir[i:i + batch_size]
        batch_images = []
        for filename in batch_files:
            filepath = os.path.join(base_dir, filename)
            img = cv2.imread(filepath) # read image
            img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA) # resize image
            batch_images.append(img)
        
        if not batch_images:
            # nothing to process in this batch
            continue
        processed_images.extend(batch_images)
    return np.array(processed_images)


def reduce_dimensions(images, reduction_method='pca', variance_threshold=0.95):
    """Apply dimension reduction using PCA or SVD"""
    original_shape = images.shape
    flattened = images.reshape(len(images), -1)
    
    if reduction_method.lower() == 'pca':
        reducer = PCA(n_components=variance_threshold)
    else:
        reducer = TruncatedSVD(n_components=min(flattened.shape))
    
    reduced = reducer.fit_transform(flattened)
    reconstructed = reducer.inverse_transform(reduced)
    variance_preserved = sum(reducer.explained_variance_ratio_) * 100
    
    print(f"Variance preserved: {variance_preserved:.2f}%")
    return reconstructed.reshape(original_shape), variance_preserved

def apply_convolution(image, kernel):
    """Apply convolution to an image"""
    if len(image.shape) == 3:
        return np.stack([convolve2d(image[:,:,c], kernel, mode='same', boundary='wrap')
                        for c in range(image.shape[2])], axis=2)
    return convolve2d(image, kernel, mode='same', boundary='wrap')


def get_convolution_kernels():
    """Return dictionary of common convolution kernels"""
    return {
        'edge_detection': np.array([[-1, -1, -1],
                                  [-1,  8, -1],
                                  [-1, -1, -1]]),
        'sobel_x': np.array([[-1, 0, 1],
                            [-2, 0, 2],
                            [-1, 0, 1]]),
        'sobel_y': np.array([[-1, -2, -1],
                            [ 0,  0,  0],
                            [ 1,  2,  1]]),
        'color_contrast': np.array([[0, -1, 0],
                                  [-1, 5, -1],
                                  [0, -1, 0]])
    }

def apply_max_pooling(image, pool_size=2):
    """Apply max pooling to an image"""
    h, w = image.shape[:2]
    new_h, new_w = h//pool_size, w//pool_size
    pooled = np.zeros((new_h, new_w) + image.shape[2:])
    
    for i in range(new_h):
        for j in range(new_w):
            pooled[i,j] = np.max(image[i*pool_size:(i+1)*pool_size, 
                                     j*pool_size:(j+1)*pool_size], axis=(0,1))
    return pooled

def apply_avg_pooling(image, pool_size=2):
    """Apply average pooling to an image"""
    h, w = image.shape[:2]
    new_h, new_w = h//pool_size, w//pool_size
    pooled = np.zeros((new_h, new_w) + image.shape[2:])
    
    for i in range(new_h):
        for j in range(new_w):
            pooled[i,j] = np.mean(image[i*pool_size:(i+1)*pool_size, 
                                      j*pool_size:(j+1)*pool_size], axis=(0,1))
    return pooled

def flatten_batch(images):
    """Flatten batch of images"""
    return images.reshape(len(images), -1)

def remove_outliers(image, method='knn', n_neighbors=5, window_size=3):
    """Remove outliers from image using specified method"""
    
    original_shape = image.shape
    h, w = image.shape[:2]
    
    # Handle different color channels separately
    if len(original_shape) == 3:
        cleaned_image = np.zeros_like(image)
        for c in range(original_shape[2]):
            cleaned_image[:,:,c] = remove_outliers(image[:,:,c], method, n_neighbors, window_size)
        return cleaned_image
    
    if method.lower() == 'knn':
        # Prepare data for KNN imputation
        X = np.zeros((h*w, window_size**2))
        for i in range(h):
            for j in range(w):
                # Extract local window
                i_start = max(0, i - window_size//2)
                i_end = min(h, i + window_size//2 + 1)
                j_start = max(0, j - window_size//2)
                j_end = min(w, j + window_size//2 + 1)
                window = image[i_start:i_end, j_start:j_end].flatten()
                # Pad if necessary
                if len(window) < window_size**2:
                    window = np.pad(window, (0, window_size**2 - len(window)), mode='edge')
                X[i*w + j] = window
        
        # Apply KNN imputation
        imputer = KNNImputer(n_neighbors=n_neighbors)
        cleaned = imputer.fit_transform(X)
        
        # Reshape back to image
        return cleaned[:,window_size**2//2].reshape(h, w)
    
    elif method.lower() == 'median':
        return median_filter(image, size=window_size)
    
    else:
        raise ValueError(f"Unknown outlier removal method: {method}")


In [ ]:
A = os.listdir('E:/@IIT_BBS/@Sem 1/ML/Final Project/P1')
len(A)

In [ ]:
demo = cv2.imread('E:/@IIT_BBS/@Sem 1/ML/Final Project/P1/' + A[0])

In [ ]:
demo.shape

In [ ]:
cv2.imwrite('demo.png',demo)


In [ ]:
batch_size = 10  # Adjust this number based on your available memory
# Use the same absolute folder that was used to create A
base_dir = r'E:/@IIT_BBS/@Sem 1/ML/Final Project/P1'



In [ ]:
process_img = preprocess_images(batch_size,base_dir)

 28%|██▊       | 12/43 [00:11<00:29,  1.05it/s]

In [ ]:
red_img,var_img =reduce_dimensions(preprocess_images(batch_size,base_dir), reduction_method='pca', variance_threshold=0.95)

In [ ]:
non_outlier_img = remove_outliers(red_img[30], method='knn', n_neighbors=5, window_size=3)

In [ ]:


cv2.imwrite("Reduced_image.png", red_img[30])
cv2.imwrite("Processed_image.png", process_img[30])
cv2.imwrite("Non_outlier_image.png", non_outlier_img)

In [ ]:


# ----------------------------------------------------------
# 1. Custom convolution application with multiple filters
# ----------------------------------------------------------
def apply_conv(image, kernel_size=3):
    """Apply a bank of convolutional filters to each input channel and stack results as a multi-channel image.

    Fix: ensure all intermediate OpenCV operations use a consistent depth (CV_32F) to avoid unsupported
    source/destination format combinations.
    """
    # Ensure list of single-channel arrays (use float32 consistently)
    if image.ndim == 2:
        channels = [image.copy().astype(np.float32)]
    else:
        channels = [image[..., c].astype(np.float32) for c in range(image.shape[2])]

    filters = []

    for ch in channels:
        # Use CV_32F for all OpenCV ops to keep consistent dtypes
        sobelx = cv2.Sobel(ch, cv2.CV_32F, 1, 0, ksize=kernel_size)
        sobely = cv2.Sobel(ch, cv2.CV_32F, 0, 1, ksize=kernel_size)
        filters.extend([sobelx, sobely])

        # Laplacian with CV_32F
        lap = cv2.Laplacian(ch, cv2.CV_32F)
        filters.append(lap)

        # Gabor filters (various orientations) - kernel as CV_32F, filter2D output CV_32F
        for theta in np.arange(0, np.pi, np.pi / 6):  # 6 orientations
            kernel = cv2.getGaborKernel((11, 11), 4.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
            fimg = cv2.filter2D(ch, cv2.CV_32F, kernel)
            filters.append(fimg)

        # Difference of Gaussians (Gaussian retains float32)
        g1 = cv2.GaussianBlur(ch, (3, 3), 1)
        g2 = cv2.GaussianBlur(ch, (5, 5), 2)
        dog = g1 - g2
        filters.append(dog)

    # Stack as multichannel image: H x W x (num_filters * num_input_channels)
    # Ensure everything is float32 before stacking
    filters = [f.astype(np.float32) for f in filters]
    stacked = np.stack(filters, axis=-1)

    # Normalize each channel to 0-1 safely
    stacked = np.moveaxis(stacked, -1, 0)
    normed = []
    for ch in stacked:
        mn = ch.min()
        mx = ch.max()
        if mx - mn < 1e-6:
            normed.append(np.zeros_like(ch))
        else:
            normed.append((ch - mn) / (mx - mn))
    stacked = np.moveaxis(np.array(normed, dtype=np.float32), 0, -1)
    return stacked

# ----------------------------------------------------------
# 2. Max pooling utility
# ----------------------------------------------------------
def apply_maxpooling(image, target_pool_size=8):
    """Apply pooling safely even when image has >4 channels, ensuring a fixed output spatial size."""
    pooled = image.copy().astype(np.float32)
    if target_pool_size is None:
        raise ValueError("target_pool_size must be provided.")

    def safe_resize(image, new_w, new_h):
        if image.ndim == 2 or image.shape[2] <= 4:
            return cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
        else:
            resized_channels = [
                cv2.resize(image[..., c], (new_w, new_h), interpolation=cv2.INTER_AREA)
                for c in range(image.shape[2])
            ]
            return np.stack(resized_channels, axis=-1)

    # Iteratively reduce size until dimensions are close to target_pool_size
    while True:
        h, w = pooled.shape[:2]
        if h <= target_pool_size and w <= target_pool_size: # Stop if both dimensions are <= target
            break
        # Reduce by half, ensuring dimensions don't become zero
        pooled = safe_resize(pooled, max(1, w // 2), max(1, h // 2))

    # Final resize to ensure exact target_pool_size x target_pool_size spatial dimensions
    # This handles cases where pooled might be (7,7) or (9,9) after the loop, for target_pool_size=8
    if pooled.shape[0] != target_pool_size or pooled.shape[1] != target_pool_size:
        pooled = safe_resize(pooled, target_pool_size, target_pool_size)

    # print("apply_maxpooling is running....")
    return pooled

# ----------------------------------------------------------
# 3. Full feature extraction pipeline
# ----------------------------------------------------------
def feature_extraction(image, kernel_size=3, target_pool_size=7):
    """Extract rich handcrafted features using convolutional filter bank and pooling."""
    conv_output = apply_conv(image, kernel_size=kernel_size)
    pooled_output = apply_maxpooling(conv_output, target_pool_size=target_pool_size)

    # Flatten pooled output
    features = pooled_output.flatten()

    # Optionally add summary stats per channel (mean, std)
    channel_means = pooled_output.mean(axis=(0, 1))
    channel_stds = pooled_output.std(axis=(0, 1))

    full_feature_vector = np.concatenate([features, channel_means, channel_stds])
    # print("feature_extraction is running....\n")
    return full_feature_vector


# feats = feature_extraction(processed_images[9], kernel_size=3,target_pool_size=8)
# print(feats.shape)


In [ ]:
def batch_CFE(batch_size,base_dir, target_size=7):
    """ Batch Custom Feature Extraction using convolutional filter bank """
    processed_images = []
    img_dir = os.listdir(base_dir)

    for i in tqdm(range(0, len(img_dir), batch_size)):
        batch_files = img_dir[i:i + batch_size]
        batch_images = []
        for filename in batch_files:
            filepath = os.path.join(base_dir, filename)
            img = cv2.imread(filepath) # read image
            img = feature_extraction(img, kernel_size=3,target_pool_size=target_size)
            batch_images.append(img)

        if not batch_images:
            # nothing to process in this batch
            continue
        processed_images.extend(batch_images)
    return np.array(processed_images)

In [ ]:
batch_features = batch_CFE(batch_size=20,base_dir=base_dir, target_size=5)
batch_features.shape